In [1]:
!pip install psycopg2-binary

In [2]:
from dotenv import load_dotenv
import psycopg2
import os

load_dotenv("./keys/postgresql.env", override=True)

POSTGRESQL_HOST = os.getenv("POSTGRESQL_HOST")
POSTGRESQL_PORT = os.getenv("POSTGRESQL_PORT")
POSTGRESQL_DB = os.getenv("POSTGRESQL_DB")
POSTGRESQL_USER = os.getenv("POSTGRESQL_USER")
POSTGRESQL_PASSWORD = os.getenv("POSTGRESQL_PASSWORD")

In [9]:
DATABASE_STRUCT_DICT = {
    "bronze_schema" :{"stock_prices_fact_table":{"columns_names": ["stock_symbol", "current_price", "change", "percent_change", "high_price_of_the_day", "low_price_of_the_day", "open_price_of_the_day", "previous_close_price"], "columns_data_types_list": ["str", "f64", "f64", "f64", "f64", "f64", "f64", "f64"]},
                      "company_information_dim_table":{"columns_names": ["symbol", "price", "beta", "volAvg", "mktCap", "lastDiv", "range", "changes", "companyName", "currency", "cik", "isin", "cusip", "exchange", "exchangeShortName", "industry", "website", "description", "ceo", "sector", "country", "fullTimeEmployees", "phone", "address", "city", "state", "zip", "dcfDiff", "dcf", "image", "ipoDate", "defaultImage", "isEtf", "isActivelyTrading", "isAdr", "isFund"], "columns_data_types_list": ["str", "f64", "f64", "i64", "i64", "f64", "str", "f64", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "f64", "f64", "str", "str", "bool", "bool", "bool", "bool", "bool"]}},
    "silver_schema" :{"stock_prices_fact_table":{"columns_names": ["stock_symbol", "current_price", "change", "percent_change", "high_price_of_the_day", "low_price_of_the_day", "open_price_of_the_day", "previous_close_price"], "columns_data_types_list": ["str", "f64", "f64", "f64", "f64", "f64", "f64", "f64"]},
                      "company_information_dim_table":{"columns_names": ["symbol", "price", "beta", "volAvg", "mktCap", "lastDiv", "range", "changes", "companyName", "currency", "cik", "isin", "cusip", "exchange", "exchangeShortName", "industry", "website", "description", "ceo", "sector", "country", "fullTimeEmployees", "phone", "address", "city", "state", "zip", "dcfDiff", "dcf", "image", "ipoDate", "defaultImage", "isEtf", "isActivelyTrading", "isAdr", "isFund"], "columns_data_types_list": ["str", "f64", "f64", "i64", "i64", "f64", "str", "f64", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "f64", "f64", "str", "str", "bool", "bool", "bool", "bool", "bool"]}},
    "gold_schema" :{"stock_prices_fact_table":{"columns_names": ["stock_symbol", "current_price", "change", "percent_change", "high_price_of_the_day", "low_price_of_the_day", "open_price_of_the_day", "previous_close_price"], "columns_data_types_list": ["str", "f64", "f64", "f64", "f64", "f64", "f64", "f64"]},
                      "company_information_dim_table":{"columns_names": ["symbol", "price", "beta", "volAvg", "mktCap", "lastDiv", "range", "changes", "companyName", "currency", "cik", "isin", "cusip", "exchange", "exchangeShortName", "industry", "website", "description", "ceo", "sector", "country", "fullTimeEmployees", "phone", "address", "city", "state", "zip", "dcfDiff", "dcf", "image", "ipoDate", "defaultImage", "isEtf", "isActivelyTrading", "isAdr", "isFund"], "columns_data_types_list": ["str", "f64", "f64", "i64", "i64", "f64", "str", "f64", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "str", "f64", "f64", "str", "str", "bool", "bool", "bool", "bool", "bool"]}}
}

DATA_TYPE_MAPPING_POLARS_TO_SQL = {
    "str": "TEXT",
    "f64": "DOUBLE PRECISION",
    "i64": "BIGINT",
    "bool": "BOOLEAN"
}

In [8]:
class PostgreSQLSetUp:
    def __init__(self, host, port, database, user, password):
        """
        Initialize the PostgreSQLSetUp with connection parameters.
        """
        self.host = host
        self.port = port
        self.database = database
        self.user = user
        self.password = password
        self.connection = None

    def connect(self):
        """
        Establish a connection to the PostgreSQL database.
        """
        try:
            self.connection = psycopg2.connect(
                host=self.host,
                port=self.port,
                database=self.database,
                user=self.user,
                password=self.password
            )
            print("Connection successful!")
        except Exception as error:
            print(f"Error connecting to PostgreSQL: {error}")

    def close(self):
        """
        Close the database connection.
        """
        if self.connection:
            self.connection.close()
            print("Connection closed.")

    def execute_sql(self, sql_script):
        """
        Execute a given SQL script.
        """
        if self.connection is None:
            print("Connection not established. Please call the connect() method first.")
            return

        try:
            cursor = self.connection.cursor()
            cursor.execute(sql_script)
            self.connection.commit()
            print("SQL script executed successfully!")
        except Exception as error:
            print(f"Error executing SQL script: {error}")
        finally:
            cursor.close()

    def create_schema(self, schema):
        """
        Create schema if it does not exist.
        """
        try:
            self.connect()
            sql_script = f"CREATE SCHEMA IF NOT EXISTS {schema};"
            self.execute_sql(sql_script)
            self.close()
            return True
        except Exception as error:
            print(f"Error creating schema: {error}")
            return False

    def create_tables(self, schema, table_name, columns_list, columns_data_types_list):
        """
        Create tables in the specified schema with given columns and data types.
        """
        try:
            self.connect()
            columns_definitions = [
                f"{column_name} {self.get_sql_data_type(data_type)}"
                for column_name, data_type in zip(columns_list, columns_data_types_list)
            ]
            columns_definitions_str = ", ".join(columns_definitions)

            sql_script = f"""
            CREATE TABLE IF NOT EXISTS {schema}.{table_name} (
                {columns_definitions_str}
            );
            """
            self.execute_sql(sql_script)
            self.close()
            print(f"Table {schema}.{table_name} created successfully!")
        except Exception as error:
            print(f"Error creating table {schema}.{table_name}: {error}")

    def get_sql_data_type(self, data_type):
        """
        Map Python data types to PostgreSQL data types.
        """
        return DATA_TYPE_MAPPING_POLARS_TO_SQL.get(data_type, "TEXT")

    def set_up(self):
        try:
            for schema, tables in DATABASE_STRUCT_DICT.items():
                if self.create_schema(schema):
                    for table_name, table_info in tables.items():
                        self.create_tables(
                            schema,
                            table_name,
                            table_info["columns_names"],
                            table_info["columns_data_types_list"]
                        )
            return True
        except Exception as error:
            print(f"Error creating table {schema}.{table_name}: {error}")
            return False

if __name__ == "__main__":
    client = PostgreSQLSetUp(POSTGRESQL_HOST, POSTGRESQL_PORT, POSTGRESQL_DB, POSTGRESQL_USER, POSTGRESQL_PASSWORD)
    result = client.set_up()
    if result:
        print("Success")

Connection successful!
SQL script executed successfully!
Connection closed.
Connection successful!
SQL script executed successfully!
Connection closed.
Table bronze_schema.stock_prices_fact_table created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Table bronze_schema.company_information_dim_table created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Connection successful!
SQL script executed successfully!
Connection closed.
Table silver_schema.stock_prices_fact_table created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Table silver_schema.company_information_dim_table created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Connection successful!
SQL script executed successfully!
Connection closed.
Table gold_schema.stock_prices_fact_table created successfully!
Connection successful!
SQL script executed successfu